In [ ]:
!pip uninstall -y deepxde && rm -rf deepxde; git clone https://github.com/g-w1/deepxde.git ; cd deepxde && git config --global user.email "a@a.com" ; git config --global user.name "a"; git checkout loss-scaling ;ls;pip install ./
!pip install matplotlib
import urllib.request
urllib.request.urlretrieve("https://github.com/lululxvi/deepxde/blob/master/examples/dataset/Burgers.npz?raw=true", "Burgers.npz")

('Burgers.npz', <http.client.HTTPMessage at 0x7d05a9822dd0>)

In [ ]:
!nvidia-smi

Sun Sep  3 20:52:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%env DDE_BACKEND=tensorflow

import deepxde as dde
assert dde.backend.tensorflow.is_gpu_available()

import numpy as np
from tensorflow.keras import mixed_precision
import tensorflow as tf
import time
def gen_testdata():
    data = np.load("Burgers.npz")
    t, x, exact = data["t"], data["x"], data["usol"].T
    xx, tt = np.meshgrid(x, t)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = exact.flatten()[:, None]
    return X, y
print('gpu:',dde.backend.tensorflow.is_gpu_available())
def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t + y * dy_x - 0.01 / np.pi * dy_xx

def burgers(float_t, mixed, loss_scaling, give_history=False):
    if mixed:
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        dde.config.set_default_float('float16')
        assert policy.compute_dtype=="float16" and policy.variable_dtype == "float32"
    elif float_t == "float16":
        policy = mixed_precision.Policy('float16')
        mixed_precision.set_global_policy(policy)
        dde.config.set_default_float('float16')
        assert policy.compute_dtype=="float16" and policy.variable_dtype == "float16"
    elif float_t == "float32":
        policy = mixed_precision.Policy('float32')
        mixed_precision.set_global_policy(policy)
        dde.config.set_default_float('float32')
        assert policy.compute_dtype=="float32" and policy.variable_dtype == "float32"
    else:
        assert False # precision needs to be 16, 32, or mixed

    start_time = time.time()
    geom = dde.geometry.Interval(-1, 1)
    timedomain = dde.geometry.TimeDomain(0, 0.99)
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)

    bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
    ic = dde.icbc.IC(
        geomtime, lambda x: -np.sin(np.pi * x[:, 0:1]), lambda _, on_initial: on_initial
    )

    data = dde.data.TimePDE(
        geomtime, pde, [bc, ic], num_domain=2048, num_boundary=64, num_initial=128, num_test=1
    )
    net = dde.nn.FNN([2] + [32] * 3 + [1], "tanh", "Glorot normal")
    model = dde.Model(data, net)
    opt = dde.optimizers.get('adam',learning_rate=1e-3)
    opt.epsilon = 1e-5
    model.compile(opt, lr=1e-3, loss_scaling=loss_scaling)
    assert model.loss_scaling == loss_scaling
    losshistory, train_state = model.train(iterations=20_000)
    memory_used = tf.config.experimental.get_memory_info('GPU:0')
    # model.compile("L-BFGS")
    # losshistory, train_state = model.train()
    dde.saveplot(losshistory, train_state, issave=False, isplot=False)
    end_time = time.time()
    elapsed_time = end_time - start_time

    X, y_true = gen_testdata()
    y_pred = model.predict(X)
    # f = model.predict(X, operator=pde)
    # print("Mean residual:", np.mean(np.absolute(f)))
    l2_relative_error = dde.metrics.l2_relative_error(y_true, y_pred)
    print("L2 relative error:", l2_relative_error)
    # np.savetxt("test.dat", np.hstack((X, y_true, y_pred)))
    print("elapsed", elapsed_time)
    print(f'memory: {memory_used}, elapsed_time: {elapsed_time}, error: {l2_relative_error}')
    if give_history:
         return losshistory, [memory_used['peak'], elapsed_time, l2_relative_error]
    else:
        return np.array([memory_used['peak'], elapsed_time, l2_relative_error])

Using backend: tensorflow



env: DDE_BACKEND=tensorflow
Enable just-in-time compilation with XLA.

gpu: True


In [ ]:
# doing a bunch of runs
def runN(float_t, mixed, loss_scaling, n):
    a = []
    for i in range(n):
        print(f'on iteration {i}')
        tf.keras.backend.clear_session()
        tf.config.experimental.reset_memory_stats('GPU:0')
        # res = dde.utils.apply(burgers, [float_t, mixed, loss_scaling])
        res = burgers(float_t, mixed, loss_scaling)
        print(f'results from apply {res}')
        a.append(res)
    return np.array(a)
    # return np.array([burgers(float_t, mixed, loss_scaling) for _ in range(10)])

In [ ]:
N = 10
mixed = runN("float16", True, False, N)
mixed_scale = runN("float16", True, True, N)
baseline_32 = runN("float32", False, False, N)
baseline_16 = runN("float16", False, False, N)



In [ ]:
def stats(array, name, idx):
    # print(name, array)
    return f"{name} &  $  \\num{{{array[:, 0].mean()}}}\\pm \\num{{{array[:, 0].std()}}} $  & $ \\num[exponent-mode = fixed, fixed-exponent = 0]{{{array[:, 1].mean()}}}\\pm \\num[exponent-mode = fixed, fixed-exponent = 0]{{{array[:, 1].std()}}} $ & $ \\num{{{array[:, 2].mean()}}}\\pm \\num{{{array[:, 2].std()}}} $ \\\\"
print(stats(mixed, "mixed", 1))
print(stats(mixed_scale, "mixed_scale", 1))
print(stats(baseline_32, "baseline_32", 1))
print(stats(baseline_16, "baseline_16", 1))
tosave = np.array([mixed,mixed_scale,baseline_32,baseline_16])
np.save("res.npy",tosave)
print("saved in order of mixed, mixed_scale, baseline_32, baseline_16")

mixed &  $  \num{14151116.8}\pm \num{29052459.7797432} $  & $ \num[exponent-mode = fixed, fixed-exponent = 0]{39.43570263385773}\pm \num[exponent-mode = fixed, fixed-exponent = 0]{2.2910849688330406} $ & $ \num{0.04508375301602895}\pm \num{0.010782991967012526} $ \\
mixed_scale &  $  \num{4558182.4}\pm \num{68131.61895507842} $  & $ \num[exponent-mode = fixed, fixed-exponent = 0]{53.158282566070554}\pm \num[exponent-mode = fixed, fixed-exponent = 0]{1.0179522962663399} $ & $ \num{0.054191315982920374}\pm \num{0.03568759630862568} $ \\
baseline_32 &  $  \num{18147353.6}\pm \num{28030645.2368766} $  & $ \num[exponent-mode = fixed, fixed-exponent = 0]{41.16562900543213}\pm \num[exponent-mode = fixed, fixed-exponent = 0]{0.5869784763468104} $ & $ \num{0.037264403411565616}\pm \num{0.009878751949832778} $ \\
baseline_16 &  $  \num{4504704.0}\pm \num{108958.01795554101} $  & $ \num[exponent-mode = fixed, fixed-exponent = 0]{38.91752686500549}\pm \num[exponent-mode = fixed, fixed-exponent = 0

In [ ]:
from scipy import stats
import numpy as np
# analyze the results
res = np.load('res.npy', allow_pickle=True)
print(np.nanmean(res, axis=1))
print(np.nanstd(res, axis=1))
# now calculate the pvalues
for i in [0, 1, 3]:
    print(['mixed', 'mixed_loss', 'ERROR', 'float16'][i], 'vs float32')
    print('memory:')
    print(stats.ttest_ind(res[2, :, 0], res[i, :, 0]).pvalue)
    print('time:')
    print(stats.ttest_ind(res[2, :, 1], res[i, :, 1]).pvalue)
    print('metric:')
    print(stats.ttest_ind(res[2, :, 2], res[i, :, 2], nan_policy='omit').pvalue)